<a href="https://colab.research.google.com/github/BradyMaes1/SearchAlgorithms/blob/main/SearchMethodsProgram1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Name: Brady Maes
Project: Program 1: Search Methods
Description: In this project, I will test and compare 5 different search algorithms for pathfinding through a small list of physical locations.
Search Algorithms: Breadth-First Search (BFS), Depth-First Search (DFS), Iterative Deepening Depth-First search (ID-DFS), Best-First search, and A* search (pronounced A star)

"""

In [82]:
from google.colab import drive
drive.mount('/content/drive')
#I have saved the files to my drive and will access them from there

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
#@title Combining Adjacencies to Reflect Symmetry

#First, I start by combining our adjacencies to be symmetrical to prevent issues later

def load_city_adjacencies_txt(adjacencies_path):
    city_map = {}
    with open(adjacencies_path, 'r') as file:
        for line in file:
            city1, city2 = line.strip().split()
            add_symmetric_connection(city_map, city1, city2)
    return city_map

def add_symmetric_connection(city_map, city1, city2):
    if city1 not in city_map:
        city_map[city1] = []
    if city2 not in city_map[city1]:
        city_map[city1].append(city2)

    if city2 not in city_map:
        city_map[city2] = []
    if city1 not in city_map[city2]:
        city_map[city2].append(city1)

adjacencies_path = '/content/drive/MyDrive/Program1SearchMethods/Adjacencies.txt'
city_adjacency_map = load_city_adjacencies_txt(adjacencies_path)

#Print the result to double check
print(city_adjacency_map)

{'Anthony': ['Bluff_City', 'Harper', 'Argonia'], 'Bluff_City': ['Anthony', 'Kiowa', 'South_Haven', 'Mayfield'], 'Kiowa': ['Bluff_City', 'Attica', 'Coldwater'], 'Attica': ['Kiowa', 'Harper', 'Medicine_Lodge'], 'Harper': ['Attica', 'Anthony'], 'Medicine_Lodge': ['Attica'], 'Augusta': ['Winfield', 'Andover', 'Emporia'], 'Winfield': ['Augusta', 'Andover'], 'Andover': ['Winfield', 'Leon', 'Towanda', 'Augusta', 'Mulvane', 'Newton'], 'Leon': ['Andover', 'Wichita'], 'Wichita': ['Leon', 'Derby'], 'Caldwell': ['South_Haven', 'Wellington', 'Argonia'], 'South_Haven': ['Caldwell', 'Bluff_City', 'Mulvane'], 'Mayfield': ['Bluff_City', 'Wellington', 'Oxford', 'Mulvane'], 'El_Dorado': ['Towanda', 'Hillsboro', 'Newton'], 'Towanda': ['El_Dorado', 'Andover'], 'Emporia': ['Augusta', 'Newton'], 'Florence': ['McPherson', 'Hutchinson'], 'McPherson': ['Florence', 'Hillsboro', 'Marion', 'Newton', 'Salina', 'Hutchinson'], 'Hillsboro': ['McPherson', 'El_Dorado', 'Lyons'], 'Greensburg': ['Coldwater'], 'Coldwater':

In [84]:
#@title Loading In Our City Coordinates to Store Locally as a Dictionary of Tuples

import csv

def load_city_coordinates(coordinates_path):
    city_coordinates = {}
    with open(coordinates_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            city = row[0]
            latitude = float(row[1])
            longitude = float(row[2])
            city_coordinates[city] = (latitude, longitude)
    return city_coordinates

coordinates_path = '/content/drive/MyDrive/Program1SearchMethods/coordinates.csv'
city_coordinates_map = load_city_coordinates(coordinates_path)

#Print the loaded city coordinates to double check
print(city_coordinates_map)


{'Abilene': (38.9220277, -97.2666667), 'Andover': (37.6868403, -97.1657752), 'Anthony': (37.1575168, -98.0728946), 'Argonia': (37.2670166, -97.7726807), 'Attica': (37.2421271, -98.2351967), 'Augusta': (37.6913277, -97.0537108), 'Bluff_City': (37.0760844, -97.8793212), 'Caldwell': (37.0346731, -97.6179436), 'Cheney': (37.632882, -97.789442), 'Clearwater': (37.5166968, -97.5325458), 'Coldwater': (37.2574937, -99.3549149), 'Derby': (37.5517122, -97.2867892), 'El_Dorado': (37.8098997, -96.8943313), 'Emporia': (38.3792991, -96.2615044), 'Florence': (38.2434672, -96.9378672), 'Greensburg': (37.6050677, -99.3005641), 'Harper': (37.2852232, -98.0368352), 'Haven': (37.9020486, -97.7926952), 'Hays': (38.88509, -99.326202), 'Hillsboro': (38.3494571, -97.2156415), 'Hutchinson': (38.0572062, -97.9414547), 'Junction_City': (39.0379342, -96.8799338), 'Kingman': (37.6480942, -98.1693967), 'Kiowa': (37.0190996, -98.4940572), 'Leon': (37.6892622, -96.7916587), 'Lyons': (38.3477177, -98.22167), 'Manhatta

In [85]:
#@title Distance Formula Functions, Currently Using Haversine Formula
from math import radians, sin, cos, sqrt, atan2


#Euclidean distance formula to calculate distance between two lat/lon coordinates
def euclidean_distance(lat1, lon1, lat2, lon2):
    return sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

"""
# Function to calculate the total distance of a given path using Euclidean distance
def calculate_total_distance(path, city_coordinates_map):
    total_distance = 0.0
    for i in range(len(path) - 1):
        city1, city2 = path[i], path[i + 1]
        lat1, lon1 = city_coordinates_map[city1]
        lat2, lon2 = city_coordinates_map[city2]
        total_distance += euclidean_distance(lat1, lon1, lat2, lon2)
    return total_distance

"""

def haversine(lat1, lon1, lat2, lon2):
    #Earth radius in kilometers
    R = 6371.0

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2]) #Converts to radians

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

#Function to calculate the total distance of a given path using the haversine formula
def calculate_total_distance(path, city_coordinates_map): #Note: This function applies harversine formula to consecutive points in a path to give a more realistic distance because you must hit every point in your path
    total_distance = 0.0
    for i in range(len(path) - 1):
        city1, city2 = path[i], path[i + 1]
        lat1, lon1 = city_coordinates_map[city1]
        lat2, lon2 = city_coordinates_map[city2]
        total_distance += haversine(lat1, lon1, lat2, lon2)
    return total_distance

In [93]:
#@title Breadth-First Search (BFS)
#FIFO, double ended queue (deque)

import time
import sys
from collections import deque

def bfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city):
    #Tracks time
    start_time = time.time()

    #Queue for BFS, stores (current_city, path)
    queue = deque([(start_city, [start_city])])

    #Set to track visited cities
    visited = set()

    #Our main loop
    while queue:
        current_city, path = queue.popleft()

        #Check if we have reached the goal
        if current_city == goal_city:
            end_time = time.time()  #Stops the timer
            execution_time = round(end_time - start_time, 12) #Rounding to 12 decimal places because such a low complexity task yields extremely small calculation times regardless of algorithm

            #Calculate total distance of the path (actual path traversal)
            total_distance = calculate_total_distance(path, city_coordinates_map)

            #Get memory usage of the queue
            memory_usage = sys.getsizeof(queue) + sys.getsizeof(visited)

            print(f"BFS Execution Time: {execution_time:.12f} seconds")
            print(f"Path taken: {' -> '.join(path)}")
            print(f"Total distance: {total_distance:.2f} km")
            print(f"Memory used: {memory_usage} bytes")
            return path  # Return the found path

        #Mark the current city as visited
        visited.add(current_city)

        #Explore neighbors
        for neighbor in city_adjacency_map.get(current_city, []):
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor]))

    #No path was found
    end_time = time.time()  # Stop the timer
    execution_time = round(end_time - start_time, 12)
    print(f"BFS Execution Time: {execution_time:.12f} seconds")
    print("No path found.")
    return None


In [94]:
#@title Depth-First Search (DFS)
#LIFO, stack

import time
import sys

def dfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city):
    #Tracks time
    start_time = time.time()

    #Stack for DFS, stores (current_city, path)
    stack = [(start_city, [start_city])]

    #Set to track visited cities
    visited = set()

    #Perform DFS
    while stack:
        current_city, path = stack.pop()

        #Checks if we have reached the goal
        if current_city == goal_city:
            end_time = time.time()  #Stops the timer
            execution_time = round(end_time - start_time, 12)

            #Calculates total distance of the path (actual path traversal)
            total_distance = calculate_total_distance(path, city_coordinates_map)

            #Gets memory usage of the stack
            memory_usage = sys.getsizeof(stack) + sys.getsizeof(visited)

            print(f"DFS Execution Time: {execution_time:.12f} seconds")
            print(f"Path taken: {' -> '.join(path)}")
            print(f"Total distance: {total_distance:.2f} km")
            print(f"Memory used: {memory_usage} bytes")
            return path  #Return the found path

        #Marks the current city as visited
        if current_city not in visited:
            visited.add(current_city)

            #Explore neighbors (in reverse order for stack behavior)
            for neighbor in reversed(city_adjacency_map.get(current_city, [])):
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor]))

    #No path was found
    end_time = time.time()  # Stop the timer
    execution_time = round(end_time - start_time, 12)
    print(f"DFS Execution Time: {execution_time:.12f} seconds")
    print("No path found.")
    return None

In [88]:
#@title Iterative Deepening Depth-First search (ID-DFS)

import time
import sys

def iddfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city, max_depth):
    #Tracks time
    start_time = time.time()

    #Iteratively increase the depth limit
    for depth in range(max_depth):
        print(f"Searching with depth limit: {depth}")
        result = dls_search(city_adjacency_map, city_coordinates_map, start_city, goal_city, depth)

        if result:
            end_time = time.time()  #Stops the timer
            execution_time = round(end_time - start_time, 12)

            #Calculates total distance of the path
            total_distance = calculate_total_distance(result, city_coordinates_map)

            #Get memory usage of the stack (estimated for this case)
            memory_usage = sys.getsizeof(result) + sys.getsizeof(city_adjacency_map) + sys.getsizeof(city_coordinates_map)

            print(f"ID-DFS Execution Time: {execution_time:.12f} seconds")
            print(f"Path taken: {' -> '.join(result)}")
            print(f"Total distance: {total_distance:.2f} km")
            print(f"Memory used: {memory_usage} bytes")
            return result

    #No path was found
    end_time = time.time()  # Stop the timer
    execution_time = round(end_time - start_time, 12)
    print(f"ID-DFS Execution Time: {execution_time:.12f} seconds")
    print("No path found.")
    return None

#Depth-Limited Search (DLS)
def dls_search(city_adjacency_map, city_coordinates_map, current_city, goal_city, depth_limit):
    # Stack for DLS: stores (current_city, path, current_depth)
    stack = [(current_city, [current_city], 0)]

    # Set to track visited cities
    visited = set()

    while stack:
        current_city, path, depth = stack.pop()

        #If we are at the goal, return the path
        if current_city == goal_city:
            return path

        #Only continue if we haven't reached the depth limit
        if depth < depth_limit:
            visited.add(current_city)

            # Explore neighbors
            for neighbor in reversed(city_adjacency_map.get(current_city, [])):
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor], depth + 1))

    return None  #No path found at this depth limit


In [89]:
#@title Best-First Search
#Heuristic: Distance from goal using a straight line
#Priority Queue, min-heap (prioritizes min distance first in our heap)

import heapq
import time
import sys
from math import sqrt

# Best-First Search (BFS with Euclidean distance heuristic)
def best_first_search(city_adjacency_map, city_coordinates_map, start_city, goal_city):
    #Timer
    start_time = time.time()

    #Priority queue for Best First, stores (heuristic, current_city, path)
    #The priority queue orders by the Euclidean distance to the goal
    priority_queue = []
    heapq.heappush(priority_queue, (euclidean_distance(city_coordinates_map[start_city][0], city_coordinates_map[start_city][1],
                                                      city_coordinates_map[goal_city][0], city_coordinates_map[goal_city][1]),
                                    start_city, [start_city]))

    #Tracks visited cities
    visited = set()

    while priority_queue:
        heuristic_value, current_city, path = heapq.heappop(priority_queue)

        #Check if we have reached the goal
        if current_city == goal_city:
            end_time = time.time()  # Stop the timer
            execution_time = round(end_time - start_time, 12)

            #Calculate total distance of the path (actual path traversal)
            total_distance = calculate_total_distance(path, city_coordinates_map)

            #Get memory usage of the priority queue
            memory_usage = sys.getsizeof(priority_queue) + sys.getsizeof(visited)

            print(f"Best-First Search Execution Time: {execution_time:.12f} seconds")
            print(f"Path taken: {' -> '.join(path)}")
            print(f"Total distance: {total_distance:.2f} km")
            print(f"Memory used: {memory_usage} bytes")
            return path  # Return the found path

        #Mark the current city as visited
        if current_city not in visited:
            visited.add(current_city)

            #Explore neighbors
            for neighbor in city_adjacency_map.get(current_city, []):
                if neighbor not in visited:
                    #Add the neighbor to the priority queue with its Euclidean distance to the goal
                    heapq.heappush(priority_queue, (euclidean_distance(city_coordinates_map[neighbor][0], city_coordinates_map[neighbor][1], city_coordinates_map[goal_city][0], city_coordinates_map[goal_city][1]),
                                                    neighbor, path + [neighbor]))

    #No path was found
    end_time = time.time()  # Stop the timer
    execution_time = round(end_time - start_time, 12)
    print(f"Best-First Search Execution Time: {execution_time:.12f} seconds")
    print("No path found.")
    return None


In [91]:
#@title A* Search

import heapq
import time
import sys
from math import sqrt

# A* Search
def a_star_search(city_adjacency_map, city_coordinates_map, start_city, goal_city):
    #Timer
    start_time = time.time()

    #Priority queue for A* search, stores (f(n), g(n), current_city, path)
    #The priority queue orders by the total cost f(n) = g(n) + h(n)
    priority_queue = []
    heapq.heappush(priority_queue, (euclidean_distance(city_coordinates_map[start_city][0], city_coordinates_map[start_city][1],
                                                      city_coordinates_map[goal_city][0], city_coordinates_map[goal_city][1]),
                                    0, start_city, [start_city]))

    #Dictionary to track the best cost to reach each city
    cost_so_far = {start_city: 0}

    #Tracks visited cities
    visited = set()


    while priority_queue:
        f_value, g_value, current_city, path = heapq.heappop(priority_queue)

        #Check if we have reached the goal
        if current_city == goal_city:
            end_time = time.time()  # Stop the timer
            execution_time = round(end_time - start_time, 12)

            #Calculate total distance of the path (actual path traversal)
            total_distance = calculate_total_distance(path, city_coordinates_map)

            #Get memory usage of the priority queue
            memory_usage = sys.getsizeof(priority_queue) + sys.getsizeof(visited)

            print(f"A* Execution Time: {execution_time:.12f} seconds")
            print(f"Path taken: {' -> '.join(path)}")
            print(f"Total distance: {total_distance:.2f} km")
            print(f"Memory used: {memory_usage} bytes")
            return path  # Return the found path

        #Mark the current city as visited
        if current_city not in visited:
            visited.add(current_city)

            #Explore neighbors
            for neighbor in city_adjacency_map.get(current_city, []):
                #Calculate the cost to reach the neighbor (g(n))
                new_cost = g_value + euclidean_distance(city_coordinates_map[current_city][0], city_coordinates_map[current_city][1],
                                                        city_coordinates_map[neighbor][0], city_coordinates_map[neighbor][1])

                #if this path is cheaper than any previously found, or the neighbor hasn't been explored yet
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    #Calculate f(n) = g(n) + h(n)
                    priority = new_cost + euclidean_distance(city_coordinates_map[neighbor][0], city_coordinates_map[neighbor][1],
                                                             city_coordinates_map[goal_city][0], city_coordinates_map[goal_city][1])
                    heapq.heappush(priority_queue, (priority, new_cost, neighbor, path + [neighbor]))

    #No path  found
    end_time = time.time()  # Stop the timer
    execution_time = round(end_time - start_time, 12)
    print(f"A* Execution Time: {execution_time:.12f} seconds")
    print("No path found.")
    return None



In [96]:
#@title Testing Our Algorithms
#Note: If we assume that our map is flat, we can simply calculate our distances using the Euclidean Distance formula. However, the use of longitude/latitude lead me to believe that we are meant to account
#      at least partially for the curvature of the earth and use the Haversine formula for calculating distances.


#Try to make sure to use example cities that aren't adjacent to one another
start_city = input('What is the start city?: ')
goal_city = input('What is the end city?: ')
print("\n\n")
#start_city = 'Anthony'
#goal_city = 'Attica'

#BFS
pathBFS = bfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city)

if pathBFS:
    print(f"Breadth First Path found: {' -> '.join(pathBFS)}")
else:
    print("No path found (Breadth First Path).")


print()

#DFS
pathDFS = dfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city)

if pathDFS:
    print(f"Depth First Path found: {' -> '.join(pathDFS)}")
else:
    print("No path found (Depth First Path).")


print()


#ID-DFS
max_depth = 5 #Define a max_depth that the algorithm will start with
pathIDDFS = iddfs_search(city_adjacency_map, city_coordinates_map, start_city, goal_city, max_depth)

if pathIDDFS:
    print(f"ID-DFS Path found: {' -> '.join(pathIDDFS)}")
else:
    print("No path found (ID-DFS Path).")


print()


#Best-First Search (Heuristic Approach #1)
pathBestFirst = best_first_search(city_adjacency_map, city_coordinates_map, start_city, goal_city)

if pathBestFirst:
    print(f"Best First Path found: {' -> '.join(pathBestFirst)}")
else:
    print("No path found (Best First Path).")


print()


#A-Star Search (Heuristic Approach #2)
pathAstar = a_star_search(city_adjacency_map, city_coordinates_map, start_city, goal_city)

if pathAstar:
    print(f"A* Path found: {' -> '.join(pathAstar)}")
else:
    print("No path found (A* Path).")


print()


What is the start city?: Kiowa
What is the end city?: Cheney



BFS Execution Time: 0.000053167343 seconds
Path taken: Kiowa -> Coldwater -> Pratt -> Cheney
Total distance: 2.59 km
Memory used: 1352 bytes
Breadth First Path found: Kiowa -> Coldwater -> Pratt -> Cheney

DFS Execution Time: 0.000040769577 seconds
Path taken: Kiowa -> Bluff_City -> Anthony -> Argonia -> Rago -> Viola -> Sawyer -> Pratt -> Cheney
Total distance: 4.18 km
Memory used: 880 bytes
Depth First Path found: Kiowa -> Bluff_City -> Anthony -> Argonia -> Rago -> Viola -> Sawyer -> Pratt -> Cheney

Searching with depth limit: 0
Searching with depth limit: 1
Searching with depth limit: 2
Searching with depth limit: 3
ID-DFS Execution Time: 0.000152826309 seconds
Path taken: Kiowa -> Coldwater -> Pratt -> Cheney
Total distance: 2.59 km
Memory used: 4632 bytes
ID-DFS Path found: Kiowa -> Coldwater -> Pratt -> Cheney

Best-First Search Execution Time: 0.000061988831 seconds
Path taken: Kiowa -> Bluff_City -> Mayfield -> M